In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import xgboost as xgb
import csv
from sklearn import preprocessing
from keras.layers import LSTM, Dense
from keras.models import Sequential
from keras.layers.core import Masking
from keras.layers.embeddings import Embedding
df1 = pd.read_csv("data/train_preliminary/user.csv")
df2 = pd.read_csv("data/train_preliminary/ad.csv")
df3 = pd.read_csv("data/train_preliminary/click_log.csv")

Using TensorFlow backend.


In [3]:
df3.sort_values(['user_id', 'time'], inplace=True)
df3

,time,user_id,creative_id,click_times
26196513,20,1,821396,1
26196514,20,1,209778,1
26196515,20,1,877468,1
26196517,39,1,1683713,1
26196524,40,1,122032,1
...,...,...,...,...
26008365,57,900000,2692144,1
26008358,71,900000,3391205,1
26008359,72,900000,3471208,1
26008361,74,900000,3527098,1


In [68]:
sentences = np.array(df3['creative_id'])
sentences.shape

(30082771,)

In [6]:
!pip install gensim
from gensim.models import Word2Vec

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 24.2 MB 648 kB/s eta 0:00:01
     |████████████████████████████████| 103 kB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.4 MB 870 kB/s eta 0:00:01
     |████████████████████████████████| 128 kB 862 kB/s eta 0:00:01
     |████████████████████████████████| 6.2 MB 737 kB/s eta 0:00:01
     |████████████████████████████████| 69 kB 924 kB/s eta 0:00:011
     |████████████████████████████████| 547 kB 739 kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-2.0.0-py3-none-any.whl size=101341 sha256=b4077b6da422f516f8dc92d03a292ede9469d9f176850a6349bc33f0b22dcd78
  Stored in directory: /Users/lixiang/Library/Caches/pip/wheels/fa/90/37/ba7001e28d41df9e51a76832d0f074e15209b45975c36fe10a
Successfully built smart-open
You should consider upgrading via the '/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/bin/python3 -m pip install --upgrade pip' command.


In [7]:
!pip install Cpython

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 346 kB 977 kB/s eta 0:00:01
  Created wheel for Cpython: filename=cPython-0.0.5-py3-none-any.whl size=4581 sha256=91efad2084a66776b380ad76d2932558e2155f4a274a9b68ff94b067682ef64d
  Stored in directory: /Users/lixiang/Library/Caches/pip/wheels/f8/4f/eb/61ee0b2a066345b007137d10de02b282d4ac33c5f29f36f4b1
Successfully built Cpython
You should consider upgrading via the '/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/bin/python3 -m pip install --upgrade pip' command.


In [69]:
sentences = list(sentences)

In [70]:
for i in range(len(sentences)):
    sentences[i] = str(sentences[i])

In [71]:
sentences = np.array(sentences).reshape(1, -1)
len(sentences)

1

In [76]:
sentences

array([['821396', '209778', '877468', ..., '3471208', '3527098',
        '4387250']], dtype='<U7')

In [80]:
model= Word2Vec(size=20, iter=1, min_count=0, window=10)
model.build_vocab(sentences)

In [87]:
model.train(sentences[0, :], total_examples=model.corpus_count, epochs=5)

(480671750, 973766880)

In [88]:
len(model.wv.vocab.keys())

2481135

In [89]:
model['821396']

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([ 0.00550138, -0.01321523, -0.02479015, -0.01082552,  0.01266956,
        0.00727372, -0.00173351, -0.01982521,  0.00812281, -0.00372631,
       -0.01859471,  0.01972486, -0.00523568, -0.01869161,  0.00740368,
        0.00153358, -0.01800693,  0.0095886 ,  0.01744165, -0.01757688],
      dtype=float32)

In [19]:
def check_matrix(x):
    if x.shape[0] >= 30:
        return x.iloc[:30, :]
    else:
        tmp1 = np.zeros((30 - x.shape[0], x.shape[1]))
        tmp1 = pd.DataFrame(tmp1)
        tmp1.columns = x.columns
        return pd.concat([x, tmp1], axis=0)
X = df3.groupby('user_id').apply(check_matrix)
X = X.reset_index(drop=True)
X = X.drop('user_id', axis = 1)
X

,time,creative_id,click_times
0,20.0,821396.0,1.0
1,20.0,209778.0,1.0
2,20.0,877468.0,1.0
3,39.0,1683713.0,1.0
4,40.0,122032.0,1.0
...,...,...,...
26999995,0.0,0.0,0.0
26999996,0.0,0.0,0.0
26999997,0.0,0.0,0.0
26999998,0.0,0.0,0.0


In [91]:
x = np.array(X)
x_tmp = np.zeros((27000000, 20))
for i in range(len(x)):
    if x[i, 1] == 0:
        continue
    x_tmp[i, :] = model[str(int(x[i, 1]))]

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [98]:
x = np.delete(x, [1], axis=1)
x.shape

(27000000, 2)

In [99]:
x = np.concatenate((x, x_tmp), axis=1)
x.shape

(27000000, 22)

In [100]:
x = x.reshape((-1, 30, 22))

In [103]:
from keras.layers.core import Masking
X_train, X_test = x[:10000, :, :], x[800000:, :, :]
Y2_train, Y2_test = df1.iloc[:10000, 2:3], df1.iloc[800000:, 2:3]
model = Sequential()
model.add(Masking(mask_value=0., input_shape=(30, 22)))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='MSE', optimizer='adam')
model.fit(X_train, Y2_train, nb_epoch=20)

/Users/lixiang/Documents/GitHub/xzwbsrz_pre/venv/lib/python3.6/site-packages/ipykernel_launcher.py:9: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if __name__ == '__main__':


Epoch 1/20
10000/10000 [==============================] - 6s 641us/step - loss: 0.2801
Epoch 2/20
10000/10000 [==============================] - 6s 618us/step - loss: 0.2214
Epoch 3/20
10000/10000 [==============================] - 7s 650us/step - loss: 0.2211
Epoch 4/20
10000/10000 [==============================] - 6s 647us/step - loss: 0.2214
Epoch 5/20
10000/10000 [==============================] - 7s 655us/step - loss: 0.2212
Epoch 6/20
10000/10000 [==============================] - 6s 637us/step - loss: 0.2211
Epoch 7/20
10000/10000 [==============================] - 6s 637us/step - loss: 0.2214
Epoch 8/20
10000/10000 [==============================] - 7s 650us/step - loss: 0.2211
Epoch 9/20
10000/10000 [==============================] - 7s 659us/step - loss: 0.2211
Epoch 10/20
10000/10000 [==============================] - 6s 648us/step - loss: 0.2212
Epoch 11/20
10000/10000 [==============================] - 6s 646us/step - loss: 0.2217
Epoch 12/20
10000/10000 [================